In [34]:
import pandas as pd
RUN_NAME = 'run 2 with 5 nodes'

In [4]:
training_data_df = pd.read_csv("sales_data_training.csv") 
test_data_df =  pd.read_csv("sales_data_test.csv") 

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

In [7]:
scaled_training_df = pd.DataFrame(scaled_training, columns = training_data_df.columns)
scaled_testing_df = pd.DataFrame(scaled_testing, columns = test_data_df.columns)

In [8]:
scaled_training_df.to_csv("sales_data_training_scaled.csv", index=False)
scaled_training_df.to_csv("sales_data_testing_scaled.csv", index=False)


In [35]:
#model
training_data_df = pd.read_csv("sales_data_training_scaled.csv")
x = training_data_df.drop('total_earnings', axis=1).values
y=training_data_df[['total_earnings']].values

from keras.models import Sequential 
from keras.layers import *
model = Sequential()
model.add(Dense(5, input_dim=9, activation='relu', name="input_layer"))
model.add(Dense(100, activation='relu', name="HL1"))
model.add(Dense(50, activation='relu', name = "HL2"))

model.add(Dense(1, activation='linear' ,name = "Out_Layer"))

model.compile(loss='mean_squared_error', optimizer='adam')

In [36]:
#tensorboard
import keras
logger = keras.callbacks.TensorBoard(log_dir='logs/{}'.format(RUN_NAME), write_graph=True, histogram_freq=5)


In [37]:
model.fit(x,y, epochs=50,shuffle=True, verbose=1, callbacks =[logger])


Epoch 1/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0526
Epoch 2/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0227
Epoch 3/50
32/32 [==============================] - 0s 984us/step - loss: 0.0154
Epoch 4/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0114
Epoch 5/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0086
Epoch 6/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0070
Epoch 7/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 8/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0042
Epoch 9/50
32/32 [==============================] - 0s 984us/step - loss: 0.0034
Epoch 10/50
32/32 [==============================] - 0s 938us/step - loss: 0.0028
Epoch 11/50
32/32 [==============================] - 0s 1ms/step - loss: 0.0022
Epoch 12/50
32/32 [==============================] - 0s 953us/step - loss: 0.0018
Epoch 13/50
32/32 [=====================

In [15]:
X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values
test_error_rate = model.evaluate(X_test, Y_test, verbose=0)


In [18]:
X_pred = pd.read_csv('proposed_new_product.csv').values

predictions = model.predict(X_pred)
predictions = predictions[0][0]

predictions = predictions + 0.1159
predictions = predictions / 0.0000036968

print(format(predictions))

270240.8671456481


In [19]:
#saving models
model.save('trained_model.h5')

In [21]:
from keras.models import load_model
model_two = load_model('trained_model.h5')
pred = model_two.predict(X_pred)
pred = pred + 0.1159
pred = pred / 0.0000036968

print(format(pred))

[[270240.88]]


In [ ]:
#exporting model to google cloud.
model_builder = tf.saved_model.builder.SavedModelBuilder("exported_model")

inputs = {
    'input': tf.saved_model.utils.build_tensor_info(model.input)
}
outputs = {
    'earnings': tf.saved_model.utils.build_tensor_info(model.output)
}

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()
